# Dependencies 

In [1]:
# Dependencies: 
import os
import pandas as pd
import numpy as np 
import pickle
import time
from bs4 import BeautifulSoup
from splinter import Browser 
import requests
from pprint import pprint

# Portland MLS 
    -Navigate to homepage
    -Collect the number of pages and number of listing per page
    -Scrape the href to all the listings shown

In [2]:
# Initialize Splinter
executable_path = {"executable_path": "../Resources/chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=True)

# Directing splinter to webpage
# portlandMLS_url = 'https://www.portlandmlsdirect.com/cgi-bin/real?pge=newsearch&state=na&widget=true&sortby=price&area=Portland&price_lo=0&price_hi=100000000&tot_bed_lo=0&tot_bath_lo=0&htype=ALL'
portlandMLS_url = "https://www.portlandmlsdirect.com/fine/real/estate/newsearch/cityname/Portland"
browser.visit(portlandMLS_url)

In [3]:
# Create variables
tracker = 0
link_list = []

# Create BeautifulSoup object
soupy = BeautifulSoup(browser.html, 'html.parser')

# Identify the number of pages of houses
number_of_pages = soupy.find(id='ia_btn_text').text
page_count = int(number_of_pages[5:])
print(page_count)

# Look going through all the pages of listings
for p in range(1, page_count):
    tracker += 1 
    
    # Identify all items in the gridview class
    viewgrid_all = soupy.find_all('div', class_='viewgrid')
    last_item = viewgrid_all[-1].get('id')
    viewgrid_count = int(last_item[8:]) + 1
    
    # Loop going through each home listing on a page
    for g in range(1, viewgrid_count):
        
        # Create BeautifulSoup object
        soupy = BeautifulSoup(browser.html, 'html.parser')
        
        # Click on each grid item
        elems = soupy.find('div', {'id':(f'viewgrid{g}')}).find('a')
        start_html = "https://www.portlandmlsdirect.com/"  
        link = start_html + elems['href']
        link_list.append(link)
    
    # Go to next page
    browser.find_by_id("ia_btn_next").click()
    print(f'Completion Percentage: {round((tracker/page_count)*100, 1)}%')
    
print(len(link_list))
print(link_list)

47
Completion Percentage: 2.1%
Completion Percentage: 4.3%
Completion Percentage: 6.4%
Completion Percentage: 8.5%
Completion Percentage: 10.6%
Completion Percentage: 12.8%
Completion Percentage: 14.9%
Completion Percentage: 17.0%
Completion Percentage: 19.1%
Completion Percentage: 21.3%
Completion Percentage: 23.4%
Completion Percentage: 25.5%
Completion Percentage: 27.7%
Completion Percentage: 29.8%
Completion Percentage: 31.9%
Completion Percentage: 34.0%
Completion Percentage: 36.2%
Completion Percentage: 38.3%
Completion Percentage: 40.4%
Completion Percentage: 42.6%
Completion Percentage: 44.7%
Completion Percentage: 46.8%
Completion Percentage: 48.9%
Completion Percentage: 51.1%
Completion Percentage: 53.2%
Completion Percentage: 55.3%
Completion Percentage: 57.4%
Completion Percentage: 59.6%
Completion Percentage: 61.7%
Completion Percentage: 63.8%
Completion Percentage: 66.0%
Completion Percentage: 68.1%
Completion Percentage: 70.2%
Completion Percentage: 72.3%
Completion Perc

In [4]:
# Check all links are unique
link_check = pd.DataFrame(link_list)
print(len(link_check[0].unique()))
print(len(link_check[0]))
link_list = list(link_check[0])

1656
1656


# Scrape Housing Data for each listing
    -address
    -price
    -bedrooms
    -bathrooms
    -square footage
    -year built
    -lot size
    -neighborhood
    -schools (elementary, middle and high)
    -county
    -city
    -zipcode

In [5]:
# Define variables
list_home_dict = []
loading_error_links = []
error_count = 0
bath = "BATH"
acres = "ACRES"
neighbor = "Neighborhood"

# Loop to iterate through all listing links
for links in link_list: 
    try:
        # Navigate to webpages for each home
        browser.visit(links)
        soup = BeautifulSoup(browser.html, 'html.parser')

        # To categorize homes with 0 bedrooms
        if bath in soup.find_all('div', class_="lineitem")[0].text:
            # Scraping primary data 
            address = soup.find('div', id="ia_address").text.replace("\n","").replace("\t  ", "")
            price = int(soup.find('div', id="ia_price").text.strip().replace('$', '').replace(',',''))
            bedrooms = 0 
            bathrooms = int(soup.find_all('div', class_="lineitem")[0].text[0])
            square_feet = int(soup.find_all('div', class_="lineitem")[1].text.replace(
                                                            'SQFT', '').replace(',', ''))
        
        else:
            # Scraping primary data
            address = soup.find('div', id="ia_address").text.replace("\n","").replace("\t  ", "")
            price = int(soup.find('div', id="ia_price").text.strip().replace('$', '').replace(',',''))
            bedrooms = int(soup.find_all('div', class_="lineitem")[0].text[0])
            bathrooms = float(soup.find_all('div', class_="lineitem")[1].text[0])
            square_feet = int(soup.find_all('div', class_="lineitem")[2].text.replace(
                                                            'SQFT', '').replace(',', ''))
        
        home_type = soup.find_all('div', id="PropDetailItem")[5].text[12:].replace('\n', '')
        built = int(soup.find_all('div', id="PropDetailItem")[3].text[12:])
        
        # Deals with pages that don't mention lot size
        if acres in soup.find_all('div', class_="lineitem")[4].text:
            lot_size = float(soup.find_all('div', class_="lineitem")[4].text.replace('ACRES', ''))

        else:
            lot_size = np.NaN 

        # Deals with pages that don't mention neighborhood
        if neighbor == soup.find_all('div', id='areaitemTitle')[3].text:
            neighborhood = soup.find_all('div', id='areaitemValue')[3].text

        else:
            neighborhood = "unknown"
            
        county = soup.find_all('div', id='areaitemValue')[0].text
        city = soup.find_all('div', id='areaitemValue')[1].text
        zipcode = soup.find_all('div', id='areaitemValue')[2].text

        # Schools data
        HS = soup.find_all('div', id="PropDetailTitle")[0].find_all_next('div')[10].text
        MS = soup.find_all('div', id="PropDetailTitle")[0].find_all_next('div')[6].text
        ES = soup.find_all('div', id="PropDetailTitle")[0].find_all_next('div')[2].text

        # Create dictionary to hold data collected
        home_dict = {
                'address':address,
                'price':price,
                'home_type':home_type,
                'bedrooms':bedrooms,
                'bathrooms':bathrooms,
                'square_feet':square_feet,
                'built':built,
                'lot_size':lot_size,
                'neighborhood':neighborhood,
                'county':county,
                'city':city,
                'zipcode':zipcode,
                'high_school':HS,
                'middle_school':MS,
                'elementary_school':ES
            }
        print(home_dict)

        # Append dictionaries to a list
        list_home_dict.append(home_dict)
    
    # Handles errors and collect links which data was able to be pulled
    except Exception as e:
        print('-------------')
        loading_error_links.append(links)
        error_count += 1
        print(error_count, e)
        
        pass

browser.quit()

{'address': '255 SW HARRISON ST 19b, Portland OR 97201', 'price': 250000, 'home_type': 'Condo - Contemporary', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 744, 'built': 1965, 'lot_size': nan, 'neighborhood': 'HARRISON WEST PSU OHSU', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97201', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Ainsworth'}
{'address': '1811 NW COUCH ST 410, Portland OR 97209', 'price': 250000, 'home_type': 'Condo - Tudor', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 519, 'built': 1916, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97209', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Chapman'}
{'address': '2211 SW PARK PL 102, Portland OR 97205', 'price': 250000, 'home_type': 'Condo - Contemporary', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 849, 'built': 1965, 'lot_size': nan, 'neighborhood': 'KINGS HILL', 'county': 'Multno

{'address': '255 SW HARRISON ST 11A, Portland OR 97201', 'price': 259900, 'home_type': 'Condo - Common Wall', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 579, 'built': 1965, 'lot_size': nan, 'neighborhood': 'DOWNTOWN PORTLAND', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97201', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Chapman'}
{'address': '1801 N Rosa Parks WAY 104, Portland OR 97217', 'price': 259900, 'home_type': 'Condo - Loft', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 549, 'built': 2020, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97217', 'high_school': 'Jefferson', 'middle_school': 'Ockley Green', 'elementary_school': 'Chief Joseph'}
{'address': '815 SE 148th AVE, Portland OR 97233', 'price': 259900, 'home_type': 'Single Family - Ranch', 'bedrooms': 2, 'bathrooms': 1.0, 'square_feet': 977, 'built': 1955, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnom

{'address': '15630 SE POWELL BLVD, Portland OR 97236', 'price': 270000, 'home_type': 'Single Family - 1 Story', 'bedrooms': 2, 'bathrooms': 1.0, 'square_feet': 1215, 'built': 1925, 'lot_size': 0.3, 'neighborhood': 'CENTENNIAL COMMUNITY ASSO', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97236', 'high_school': 'Centennial', 'middle_school': 'Centennial', 'elementary_school': 'Pleasant Valley'}
{'address': '4903 SE 64TH AVE, Portland OR 97206', 'price': 270000, 'home_type': 'Single Family - 1 Story', 'bedrooms': 2, 'bathrooms': 1.0, 'square_feet': 680, 'built': 1908, 'lot_size': 0.1, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97206', 'high_school': 'Franklin', 'middle_school': 'Arleta', 'elementary_school': 'Arleta'}
{'address': '3906 SE 102ND AVE, Portland OR 97266', 'price': 270000, 'home_type': 'Single Family - Ranch', 'bedrooms': 1, 'bathrooms': 3.0, 'square_feet': 1957, 'built': 1957, 'lot_size': 0.18, 'neighborhood': 'unknown', 'coun

{'address': '9183 NW GERMANTOWN RD, Portland OR 97231', 'price': 280330, 'home_type': 'Condo - Craftsman', 'bedrooms': 3, 'bathrooms': 3.0, 'square_feet': 1378, 'built': 2005, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97231', 'high_school': 'Lincoln', 'middle_school': 'Skyline', 'elementary_school': 'Skyline'}
{'address': '1500 SW SKYLINE BLVD #4, Portland OR 97221', 'price': 282500, 'home_type': 'Condo - Contemporary', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1270, 'built': 1966, 'lot_size': nan, 'neighborhood': 'Skyline Condominium', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97221', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Ainsworth'}
{'address': '3866 SE TAYLOR ST 201, Portland OR 97214', 'price': 284900, 'home_type': 'Condo - Contemporary', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 741, 'built': 1961, 'lot_size': nan, 'neighborhood': 'HAWTHORNE/BELMONT', 'county

{'address': '12637 SE BOISE ST, Portland OR 97236', 'price': 295000, 'home_type': 'Condo - Bungalow', 'bedrooms': 2, 'bathrooms': 1.0, 'square_feet': 1927, 'built': 1927, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97236', 'high_school': 'David Douglas', 'middle_school': 'Alice Ott', 'elementary_school': 'Gilbert Hts'}
{'address': '730 NW NAITO PKWY E5, Portland OR 97209', 'price': 295000, 'home_type': 'Condo - Common Wall', 'bedrooms': 1, 'bathrooms': 1.0, 'square_feet': 748, 'built': 1982, 'lot_size': nan, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97209', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Chapman'}
{'address': '3642 SW BAIRD ST 6, Portland OR 97219', 'price': 295394, 'home_type': 'Condo - Tudor', 'bedrooms': 2, 'bathrooms': 2.0, 'square_feet': 1165, 'built': 2008, 'lot_size': nan, 'neighborhood': 'EASTVIEW RIDGE CONDOMINIU', 'county': 'Multnomah

...

{'address': '841 S GAINES ST 1906, Portland OR 97239', 'price': 1495000, 'home_type': 'Condo - Contemporary', 'bedrooms': 2, 'bathrooms': 3.0, 'square_feet': 1896, 'built': 2007, 'lot_size': nan, 'neighborhood': 'SOUTH WATERFRONT ATWATER ', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97239', 'high_school': 'Wilson', 'middle_school': 'Jackson', 'elementary_school': 'Capitol Hill'}
{'address': '1908 NE 24TH AVE, Portland OR 97212', 'price': 1495000, 'home_type': 'Single Family - Craftsman', 'bedrooms': 5, 'bathrooms': 4.0, 'square_feet': 5413, 'built': 1912, 'lot_size': 0.23, 'neighborhood': 'IRVINGTON', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97212', 'high_school': 'Grant', 'middle_school': 'Harriet Tubman', 'elementary_school': 'Irvington'}
{'address': '3141 NE US GRANT PL, Portland OR 97212', 'price': 1495000, 'home_type': 'Single Family - English', 'bedrooms': 4, 'bathrooms': 3.0, 'square_feet': 4233, 'built': 1928, 'lot_size': 0.16, 'neighborhood': 'Grant P

{'address': '3241 NE 21ST AVE, Portland OR 97212', 'price': 1595000, 'home_type': 'Single Family - FourSquare', 'bedrooms': 5, 'bathrooms': 5.0, 'square_feet': 7059, 'built': 1911, 'lot_size': 0.23, 'neighborhood': 'IRVINGTON', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97212', 'high_school': 'Jefferson', 'middle_school': 'Harriet Tubman', 'elementary_school': 'Sabin'}
{'address': '1306 S MILITARY RD, Portland OR 97219', 'price': 1595000, 'home_type': 'Single Family - Traditional', 'bedrooms': 4, 'bathrooms': 4.0, 'square_feet': 4924, 'built': 2001, 'lot_size': 1.22, 'neighborhood': 'unknown', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97219', 'high_school': 'Riverdale', 'middle_school': 'Riverdale', 'elementary_school': 'Riverdale'}
-------------
33 invalid literal for int() with base 10: '1400000 - 1600000'
{'address': '7809 NE EVERETT ST, Portland OR 97213', 'price': 1600000, 'home_type': 'Single Family - Craftsman', 'bedrooms': 2, 'bathrooms': 4.0, 'square_f

{'address': '4921 NW SKYLINE BLVD, Portland OR 97229', 'price': 1799000, 'home_type': 'Single Family - Traditional', 'bedrooms': 7, 'bathrooms': 8.0, 'square_feet': 9389, 'built': 2007, 'lot_size': 2.0, 'neighborhood': 'SKYLINE / FOREST PARK', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97229', 'high_school': 'Lincoln', 'middle_school': 'Skyline', 'elementary_school': 'Skyline'}
{'address': '1920 S RIVER DR 1202, Portland OR 97201', 'price': 1799900, 'home_type': 'Condo - Contemporary', 'bedrooms': 2, 'bathrooms': 3.0, 'square_feet': 2765, 'built': 2006, 'lot_size': nan, 'neighborhood': 'STRAND, RIVERPLACE WATERF', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97201', 'high_school': 'Lincoln', 'middle_school': 'Stoller', 'elementary_school': 'Ainsworth'}
{'address': '1420 NW LOVEJOY ST 708, Portland OR 97209', 'price': 1800000, 'home_type': 'Condo - Loft', 'bedrooms': 3, 'bathrooms': 3.0, 'square_feet': 2823, 'built': 1910, 'lot_size': nan, 'neighborhood': 'Marshall

{'address': '4318 SW FAIRVIEW CIRCUS , Portland OR 97221', 'price': 2200000, 'home_type': 'Single Family - Traditional', 'bedrooms': 4, 'bathrooms': 5.0, 'square_feet': 4817, 'built': 1965, 'lot_size': 3.06, 'neighborhood': 'SW Portland', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97221', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Ainsworth'}
{'address': '2030 S MILITARY RD, Portland OR 97219', 'price': 2200000, 'home_type': 'Single Family - Saltbox', 'bedrooms': 6, 'bathrooms': 6.0, 'square_feet': 7805, 'built': 1927, 'lot_size': 1.31, 'neighborhood': 'DUNTHORPE', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97219', 'high_school': 'Riverdale', 'middle_school': 'Riverdale', 'elementary_school': 'Riverdale'}
{'address': '2600 SW GEORGIAN PL, Portland OR 97201', 'price': 2245000, 'home_type': 'Single Family - Traditional', 'bedrooms': 5, 'bathrooms': 6.0, 'square_feet': 6638, 'built': 1923, 'lot_size': 0.48, 'neighborhood': 'Port

{'address': '2425 S MILITARY RD, Portland OR 97219', 'price': 3950000, 'home_type': 'Single Family - Traditional', 'bedrooms': 5, 'bathrooms': 1.0, 'square_feet': 8580, 'built': 1938, 'lot_size': 1.23, 'neighborhood': 'DUNTHORPE', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97219', 'high_school': 'Riverdale', 'middle_school': 'Riverdale', 'elementary_school': 'Riverdale'}
{'address': '1001 NW Lovejoy ST PH-2, Portland OR 97209', 'price': 3995000, 'home_type': 'Condo - Contemporary', 'bedrooms': 3, 'bathrooms': 4.0, 'square_feet': 3495, 'built': 2006, 'lot_size': nan, 'neighborhood': 'Metropolitan', 'county': 'Multnomah', 'city': 'Portland', 'zipcode': '97209', 'high_school': 'Lincoln', 'middle_school': 'West Sylvan', 'elementary_school': 'Chapman'}
{'address': '13150 SW IRON MOUNTAIN BLVD, Portland OR 97219', 'price': 3995000, 'home_type': 'Single Family - Craftsman', 'bedrooms': 4, 'bathrooms': 6.0, 'square_feet': 6923, 'built': 2009, 'lot_size': 1.4, 'neighborhood': 'DUNTH

In [6]:
# Create a dataframe
housing_data_df = pd.DataFrame(list_home_dict)
housing_data_df.drop_duplicates(inplace=True)
housing_data_df.head()

,address,price,home_type,bedrooms,bathrooms,square_feet,built,lot_size,neighborhood,county,city,zipcode,high_school,middle_school,elementary_school
0,"255 SW HARRISON ST 19b, Portland OR 97201",250000,Condo - Contemporary,1,1.0,744,1965,NaN,HARRISON WEST PSU OHSU,Multnomah,Portland,97201,Lincoln,West Sylvan,Ainsworth
1,"1811 NW COUCH ST 410, Portland OR 97209",250000,Condo - Tudor,1,1.0,519,1916,NaN,unknown,Multnomah,Portland,97209,Lincoln,West Sylvan,Chapman
2,"2211 SW PARK PL 102, Portland OR 97205",250000,Condo - Contemporary,1,1.0,849,1965,NaN,KINGS HILL,Multnomah,Portland,97205,Lincoln,West Sylvan,Chapman
3,"501 NE Bridgeton RD 5, Portland OR 97211",250000,Floating Home - 1 Story,2,1.0,791,1964,NaN,BRIDGETON,Multnomah,Portland,97211,Jefferson,Other,Faubion
4,"8712 N DECATUR ST 301, Portland OR 97203",252000,Condo - Contemporary,2,1.0,934,2007,NaN,unknown,Multnomah,Portland,97203,Roosevelt,George,James John


In [7]:
# Check data collected verse data missed
print(f'Listings: {len(housing_data_df)}')
print(f'Missed Listings: {len(loading_error_links)}')

Listings: 1621
Missed Listings: 34


# Save Data

In [8]:
# Saving housing data
housing_data_df.to_csv("../Resources/housingData.csv", index = False, header = True)

# Saving list of housing links
with open("../Resources/housing_links.txt", "wb") as fp:   #Pickling
    pickle.dump(link_list, fp)